In [ ]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 13.9 MB/s 
     |████████████████████████████████| 596 kB 67.0 MB/s 
     |████████████████████████████████| 6.6 MB 46.6 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


https://huggingface.co/course/chapter1/3?fw=pt

In [ ]:
import numpy as np
#possibly replace with e.g. util.pytorch_cos_sim from sentence_transformers
from scipy.spatial.distance import cosine

from transformers import pipeline


In [ ]:
#this will download the default gpt2 model, around 1.2GB (or just 530MB??)
generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to create a complex virtual machine with the Python libraries and tools (pyc-python, sys-python, etc), as well as Python-based programming, in the virtual environment. Our first virtual machine'}]

Mask filling

In [ ]:
unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

[{'score': 0.19619759917259216,
  'sequence': 'This course will teach you all about mathematical models.',
  'token': 30412,
  'token_str': ' mathematical'},
 {'score': 0.04052729904651642,
  'sequence': 'This course will teach you all about computational models.',
  'token': 38163,
  'token_str': ' computational'}]

### Tokenizer and models

DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of BERT’s performances as measured on the GLUE language understanding benchmark.

https://huggingface.co/docs/transformers/model_doc/distilbert

The abstract from the paper is the following:

> As Transfer Learning from large-scale pre-trained models becomes more prevalent in Natural Language Processing (NLP), operating these large models in on-the-edge and/or under constrained computational training or inference budgets remains challenging. In this work, we propose a method to pre-train a smaller general-purpose language representation model, called DistilBERT, which can then be fine-tuned with good performances on a wide range of tasks like its larger counterparts. While most prior work investigated the use of distillation for building task-specific models, we leverage knowledge distillation during the pretraining phase and show that it is possible to reduce the size of a BERT model by 40%, while retaining 97% of its language understanding capabilities and being 60% faster. To leverage the inductive biases learned by larger models during pretraining, we introduce a triple loss combining language modeling, distillation and cosine-distance losses. Our smaller, faster and lighter model is cheaper to pre-train and we demonstrate its capabilities for on-device computations in a proof-of-concept experiment and a comparative on-device study.

In [ ]:
from transformers import AutoTokenizer

#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


In [ ]:
#What about slang akronyms ?
IDs = tokenizer(["OMG, this movie", "WTF is going"], padding=True, truncation=True, return_tensors="tf")
print(IDs)
print(tokenizer.tokenize("OMG"))
print(tokenizer.tokenize("LOL"))
print(tokenizer.tokenize("BERT"))
print(tokenizer.tokenize("Interdisciplinary"))
print(tokenizer.tokenize("comprehensibility"))

#notice that the full BERT tokenizer would split the following into sub words:
print(tokenizer.tokenize("transformers"))

{'input_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[  101, 18168,  2290,  1010,  2023,  3185,   102],
       [  101,  1059, 24475,  2003,  2183,   102,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0]], dtype=int32)>}
['om', '##g']
['lo', '##l']
['bert']
['interdisciplinary']
['com', '##pre', '##hen', '##sibility']
['transformers']


Notice the identical tokens for identical words

In [ ]:
raw_inputs = [
    "How does the dimension of the embedding come about",
    "How do you come around?",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[ 101, 2129, 2515, 1996, 9812, 1997, 1996, 7861, 8270, 4667, 2272,
        2055,  102],
       [ 101, 2129, 2079, 2017, 2272, 2105, 1029,  102,    0,    0,    0,
           0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]], dtype=int32)>}


Notice the different ways of calling the tokenizer:

In [ ]:
s = "BIPM alumni are in high demand!"
tokens = tokenizer.tokenize(s)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

ids2 = tokenizer(s, padding=True, truncation=True, return_tensors="tf")
print(ids2)

[12170, 9737, 9441, 2024, 1999, 2152, 5157, 999]
{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  101, 12170,  9737,  9441,  2024,  1999,  2152,  5157,   999,
          102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


#### Decoding tokens

Note that the decode method not only converts the indices back to tokens, but also groups together the tokens that were part of the same words to produce a readable sentence. 

In [ ]:
decoded_string = tokenizer.decode(ids)
print(decoded_string)

bipm alumni are in high demand!


### Embedding

In [ ]:
from transformers import TFAutoModel

#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = "bert-base-uncased"
model = TFAutoModel.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 13, 768)


So the output is high-dimensional (embedding of dimension 768 for each individual word!)

Let's find out if the embedding is really contextual, i.e. whether identical single words are different depending on the sentence:

In [ ]:

How_embed_1 = outputs.last_hidden_state[0,0,:]
How_embed_2 = outputs.last_hidden_state[1,0,:]
print(np.sum(How_embed_1==How_embed_2))#0
#outputs.last_hidden_state[0:2,0,0:10]


print("similarity:")
print( 1 - cosine(How_embed_1, How_embed_2) ) 
#most_similar=data_

0
similarity:
0.9125811457633972


### Tasks:

1. Embedding Similarity 
  * Write a function that takes as input two sentences, finds identical words, computes their contextual embeddings and prints the cosine similarity.
  * Ask interesting questions (case, punctuation,...)
  * Use longer words and find similarity between sub words

2. Load the IMBD reviews and sample 500 positive and negative reviews. 
  * Extract the embedding vectors
  * Fit a Naive Bayes classifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pickle
import re

# Import dataset
TWSM_path="/content/drive/MyDrive/TWSM/"
decoded_reviews = pickle.load(open(TWSM_path + "data/IMDB/decoded_reviews_train.pkl", "rb"))
train_labels = pickle.load(open(TWSM_path + "data/IMDB/train_labels.pkl", "rb"))


Mounted at /content/drive


### Task 1.1.1


In [ ]:
def find_similarity(sentence1,sentence2):
  raw_inputs = [sentence1,sentence2]
  inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")

  sentence1_split = re.sub(r"[,.;@#?!&$]+\ *", " ",sentence1.lower()).split()
  sentence2_split = re.sub(r"[,.;@#?!&$]+\ *", " ",sentence2.lower()).split()

  xy, index1, index2 = np.intersect1d(sentence1_split, sentence2_split, return_indices=True)

  outputs = model(inputs)

  for i,a in enumerate(index1):
    print(sentence1_split[index1[i]])
    How_embed_1 = outputs.last_hidden_state[0,index1[i],:]
    How_embed_2 = outputs.last_hidden_state[1,index2[i],:]
    print(np.sum(How_embed_1==How_embed_2))

    print("similarity:")
    print( 1 - cosine(How_embed_1, How_embed_2)) 

In [ ]:
raw_inputs = decoded_reviews[:2]
find_similarity(raw_inputs[0],raw_inputs[1])

a
0
similarity:
0.3012385368347168
all
0
similarity:
0.525002658367157
an
0
similarity:
0.2968720495700836
and
0
similarity:
0.29949304461479187
are
0
similarity:
0.31733086705207825
at
0
similarity:
0.3642032742500305
be
0
similarity:
0.3512457013130188
big
0
similarity:
0.20100535452365875
but
0
similarity:
0.31710559129714966
end
0
similarity:
0.30485373735427856
film
0
similarity:
0.24475358426570892
for
0
similarity:
0.3276773989200592
good
0
similarity:
0.2759435772895813
have
0
similarity:
0.3527619540691376
i
0
similarity:
0.3656502962112427
is
0
similarity:
0.353829562664032
just
0
similarity:
0.32063615322113037
of
0
similarity:
0.3231599032878876
out
0
similarity:
0.26487547159194946
so
0
similarity:
0.34995248913764954
that
0
similarity:
0.18452396988868713
the
0
similarity:
0.32059869170188904
them
0
similarity:
0.3380131423473358
there
0
similarity:
0.3873777687549591
these
0
similarity:
0.3311106860637665
this
0
similarity:
0.15176504850387573
to
0
similarity:
0.27179196

In [ ]:
raw_inputs = [
    "work to live",
    "live to work",
]
find_similarity(raw_inputs[0],raw_inputs[1])

live
0
similarity:
0.3014388084411621
to
0
similarity:
0.586905837059021
work
0
similarity:
0.13984414935112


### Task 1.1.2

In [ ]:
raw_inputs = [
    "Hast du mal ein Euro?",
    "Du hast mich gefragt.",
]
find_similarity(raw_inputs[0],raw_inputs[1])

du
0
similarity:
0.5921092629432678
hast
0
similarity:
0.4238847494125366


In [ ]:
raw_inputs = [
    "This is a very interesting sentence.",
    "Do you think that that was an interesting sentence?",
]
find_similarity(raw_inputs[0],raw_inputs[1])

interesting
0
similarity:
0.6154913902282715
sentence
0
similarity:
0.8429889678955078


### Task 1.1.3

In [ ]:
raw_inputs = [
    "As he crossed toward the pharmacy at the corner he involuntarily turned his head because of a burst of light that had ricocheted from his temple, and saw, with that quick smile with which we greet a rainbow or a rose, a blindingly white parallelogram of sky being unloaded from the van—a dresser with mirrors across which, as across a cinema screen, passed a flawlessly clear reflection of boughs sliding and swaying not arboreally, but with a human vacillation, produced by the nature of those who were carrying this sky, these boughs, this gliding façade.",
    "On offering to help the blind man, the man who then stole his car, had not, at that precise moment, had any evil intention, quite the contrary, what he did was nothing more than obey those feelings of generosity and altruism which, as everyone knows, are the two best traits of human nature and to be found in much more hardened criminals than this one, a simple car-thief without any hope of advancing in his profession, exploited by the real owners of this enterprise, for it is they who take advantage of the needs of the poor.",
]
find_similarity(raw_inputs[0],raw_inputs[1])

a
0
similarity:
0.25932368636131287
and
0
similarity:
0.38617679476737976
as
0
similarity:
0.17604389786720276
at
0
similarity:
0.24461729824543
by
0
similarity:
0.33729279041290283
had
0
similarity:
0.34519174695014954
he
0
similarity:
0.19789671897888184
his
0
similarity:
0.32391875982284546
human
0
similarity:
0.24000805616378784
nature
0
similarity:
0.38258516788482666
not
0
similarity:
0.2140049785375595
of
0
similarity:
0.4634685218334198
that
0
similarity:
0.322934627532959
the
0
similarity:
0.4095923602581024
this
0
similarity:
0.2627895176410675
those
0
similarity:
0.4039608836174011
which
0
similarity:
0.16146516799926758
who
0
similarity:
0.283297598361969


### Task 1.2

In [ ]:
# get input
positive_raw_input = list(np.array(decoded_reviews)[train_labels==1][:100])
negative_raw_input = list(np.array(decoded_reviews)[train_labels==0][:100])

In [ ]:
positive_inputs = tokenizer(positive_raw_input, padding=True, truncation=True, return_tensors="tf")
negative_inputs = tokenizer(negative_raw_input, padding=True, truncation=True, return_tensors="tf")

In [ ]:
# positive_outputs = model(positive_inputs)

In [ ]:
negative_inputs[0]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
outputs = model(negative_inputs)

In [25]:
outputs2 = model(positive_inputs)

In [35]:
neg_out = outputs.last_hidden_state[:, 0, :].numpy()
pos_out = outputs2.last_hidden_state[:, 0, :].numpy()

In [46]:
all_out_train = np.concatenate((neg_out[:50],pos_out[:50]))
all_out_test = np.concatenate((neg_out[50:],pos_out[50:]))

In [47]:
from sklearn.naive_bayes import GaussianNB

In [48]:
y_train = np.concatenate((np.zeros(50),np.ones(50)))
y_test = np.concatenate((np.zeros(50),np.ones(50)))

In [49]:
clf = GaussianNB()
clf.fit(all_out_train, y_train)

GaussianNB()

In [50]:
clf.score(all_out_train, y_train)

0.87

In [51]:
clf.score(all_out_test, y_test)

0.68